Set up the Notebook, Load Data, and Initial Inspection

In [2]:
import pandas as pd
import os
import numpy as np # Import numpy for NaN handling

# Define the path to your raw data directory
# Adjust this path if your notebook is not in 'notebooks/' relative to 'data/'
raw_data_path = '../data/raw/'

# List the CSV files you want to load
file_names = ['coin_gecko_2022-03-16.csv', 'coin_gecko_2022-03-17.csv']

# Create an empty list to hold dataframes
dfs = []

# Load each CSV file
for file_name in file_names:
    file_path = os.path.join(raw_data_path, file_name)
    try:
        df = pd.read_csv(file_path)
        dfs.append(df)
        print(f"Successfully loaded {file_name}")
    except FileNotFoundError:
        print(f"Error: {file_name} not found at {file_path}")
    except Exception as e:
        print(f"Error loading {file_name}: {e}")

# Concatenate all dataframes into a single dataframe
if dfs:
    raw_df = pd.concat(dfs, ignore_index=True)
    print("\nCombined DataFrame created.")
    print(f"Total rows in combined DataFrame: {raw_df.shape[0]}")
else:
    raw_df = pd.DataFrame()
    print("No dataframes were loaded to combine.")

# --- Initial Data Inspection ---

print("\n--- First 5 rows of the combined data ---")
print(raw_df.head())

print("\n--- Data Info (Data Types and Non-Null Counts) ---")
print(raw_df.info())

print("\n--- Basic Statistical Summary ---")
print(raw_df.describe())

print("\n--- Missing Values Count per Column ---")
print(raw_df.isnull().sum())

Successfully loaded coin_gecko_2022-03-16.csv
Successfully loaded coin_gecko_2022-03-17.csv

Combined DataFrame created.
Total rows in combined DataFrame: 1000

--- First 5 rows of the combined data ---
       coin symbol         price     1h    24h     7d    24h_volume  \
0   Bitcoin    BTC  40859.460000  0.022  0.030  0.055  3.539076e+10   
1  Ethereum    ETH   2744.410000  0.024  0.034  0.065  1.974870e+10   
2    Tether   USDT      1.000000 -0.001 -0.001  0.000  5.793497e+10   
3       BNB    BNB    383.430000  0.018  0.028  0.004  1.395854e+09   
4  USD Coin   USDC      0.999874 -0.001  0.000 -0.000  3.872274e+09   

        mkt_cap        date  
0  7.709915e+11  2022-03-16  
1  3.271044e+11  2022-03-16  
2  7.996516e+10  2022-03-16  
3  6.404382e+10  2022-03-16  
4  5.222214e+10  2022-03-16  

--- Data Info (Data Types and Non-Null Counts) ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 9 columns):
 #   Column      Non-Null Count  

Date Column Processing and Initial Cleaning

In [7]:
# --- IMPORTANT: Replace 'Date_Column_Name' and 'Price_Column_Name' with actual column names from your data ---

# Make a copy to work on, preserving the raw_df
df = raw_df.copy()

# 2.1 Convert 'Date' column to datetime and set as index
date_column_name = 'date' # <--- REPLACE WITH YOUR ACTUAL DATE COLUMN NAME (e.g., 'Date', 'timestamp')

if date_column_name in df.columns:
    df[date_column_name] = pd.to_datetime(df[date_column_name], errors='coerce')
    # Drop rows where date conversion failed (if any, these will turn into NaT - Not a Time)
    df.dropna(subset=[date_column_name], inplace=True)
    # Sort by date to ensure chronological order for time series analysis
    df.sort_values(by=date_column_name, inplace=True)
    # Set the date column as the DataFrame index
    df.set_index(date_column_name, inplace=True)
    print(f"\n'{date_column_name}' column converted to datetime and set as index.")
else:
    print(f"Warning: '{date_column_name}' not found. Please verify the date column name from Step 1 output.")

# 2.2 Identify Price Column and ensure it's numeric
price_column_name = 'price' # <--- REPLACE WITH YOUR ACTUAL PRICE COLUMN NAME (e.g., 'Close', 'priceUsd')

if price_column_name in df.columns:
    # Coerce errors will turn non-numeric values into NaN, which we handle next
    df[price_column_name] = pd.to_numeric(df[price_column_name], errors='coerce')
    print(f"'{price_column_name}' column converted to numeric.")
else:
    print(f"Error: '{price_column_name}' column not found. Please verify the price column name from Step 1 output. This is our target column!")

# 2.3 Handle Duplicate Rows (if any)
initial_rows = df.shape[0]
df.drop_duplicates(inplace=True)
if df.shape[0] < initial_rows:
    print(f"Removed {initial_rows - df.shape[0]} duplicate rows.")
else:
    print("No duplicate rows found.")

# Display info after initial processing
print("\n--- DataFrame Info after Date Processing and Duplicates Handling ---")
print(df.info())
print("\n--- Missing Values after Initial Cleaning ---")
print(df.isnull().sum())


'date' column converted to datetime and set as index.
'price' column converted to numeric.
Removed 2 duplicate rows.

--- DataFrame Info after Date Processing and Duplicates Handling ---
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 998 entries, 2022-03-16 to 2022-03-17
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   coin        998 non-null    object 
 1   symbol      998 non-null    object 
 2   price       998 non-null    float64
 3   1h          993 non-null    float64
 4   24h         993 non-null    float64
 5   7d          992 non-null    float64
 6   24h_volume  993 non-null    float64
 7   mkt_cap     998 non-null    float64
dtypes: float64(6), object(2)
memory usage: 70.2+ KB
None

--- Missing Values after Initial Cleaning ---
coin          0
symbol        0
price         0
1h            5
24h           5
7d            6
24h_volume    5
mkt_cap       0
dtype: int64


Handle Missing Values

In [8]:
# Identify numerical columns for imputation (excluding the datetime index)
numerical_cols = df.select_dtypes(include=['number']).columns.tolist()

print(f"\nNumerical columns identified for imputation: {numerical_cols}")

# Strategy 1: Forward-fill (ffill) for time-series data
print("\nFilling missing values using forward fill (ffill)...")
df[numerical_cols] = df[numerical_cols].ffill()

# Strategy 2: Backward-fill (bfill) for any remaining NaNs (e.g., at the very beginning of the series)
print("Filling any remaining missing values using backward fill (bfill)...")
df[numerical_cols] = df[numerical_cols].bfill()

# Special handling for the target column ('price_column_name') if it still has NaNs
# This can happen if the entire series or a block at the very start/end was missing for price
if df[price_column_name].isnull().any():
    print(f"\nWarning: Some missing values remain in '{price_column_name}' even after ffill and bfill.")
    # Option: Drop rows where the target price is still NaN (recommended for target variable)
    missing_price_rows_before_drop = df[price_column_name].isnull().sum()
    df.dropna(subset=[price_column_name], inplace=True)
    print(f"Dropped {missing_price_rows_before_drop} rows with remaining missing '{price_column_name}' values.")
else:
    print(f"No missing values found in '{price_column_name}' after imputation.")


print("\n--- Missing Values Count after Imputation ---")
print(df.isnull().sum())
print(f"DataFrame shape after imputation and dropping price NaNs: {df.shape}")


Numerical columns identified for imputation: ['price', '1h', '24h', '7d', '24h_volume', 'mkt_cap']

Filling missing values using forward fill (ffill)...
Filling any remaining missing values using backward fill (bfill)...
No missing values found in 'price' after imputation.

--- Missing Values Count after Imputation ---
coin          0
symbol        0
price         0
1h            0
24h           0
7d            0
24h_volume    0
mkt_cap       0
dtype: int64
DataFrame shape after imputation and dropping price NaNs: (998, 8)


 Basic Feature Engineering (Time-based)

In [9]:
# Ensure the index is datetime for feature extraction
if not pd.api.types.is_datetime64_any_dtype(df.index):
    print("\nError: DataFrame index is not datetime. Please ensure Step 2 was completed correctly.")
else:
    print("\nGenerating time-based features...")
    df['year'] = df.index.year
    df['month'] = df.index.month
    df['day_of_week'] = df.index.dayofweek # Monday=0, Sunday=6
    df['day_of_year'] = df.index.dayofyear
    # .isocalendar().week returns a Series, need to convert to int if not already
    df['week_of_year'] = df.index.isocalendar().week.astype(int)
    df['quarter'] = df.index.quarter
    print("Time-based features created: year, month, day_of_week, day_of_year, week_of_year, quarter.")

print("\n--- First 5 rows with new time-based features ---")
print(df.head())


Generating time-based features...
Time-based features created: year, month, day_of_week, day_of_year, week_of_year, quarter.

--- First 5 rows with new time-based features ---
                      coin  symbol         price     1h    24h     7d  \
date                                                                    
2022-03-16         Bitcoin     BTC  4.085946e+04  0.022  0.030  0.055   
2022-03-16  Iron Bank EURO   IBEUR  1.080000e+00  0.000 -0.004  0.009   
2022-03-16       Prometeus    PROM  7.960000e+00  0.017  0.008  0.015   
2022-03-16    MaidSafeCoin    MAID  2.949200e-01  0.023  0.010  0.045   
2022-03-16    Bezoge Earth  BEZOGE  3.051000e-09  0.012 -0.005 -0.041   

              24h_volume       mkt_cap  year  month  day_of_week  day_of_year  \
date                                                                            
2022-03-16  3.539076e+10  7.709915e+11  2022      3            2           75   
2022-03-16  9.525810e+04  1.300442e+08  2022      3            2    

Normalize/Scale Numerical Features

In [10]:
from sklearn.preprocessing import MinMaxScaler

# Identify features to scale (all numerical columns except the target and time-based features)
# Ensure price_column_name is defined from earlier steps
features_to_scale = [col for col in df.select_dtypes(include=['number']).columns if col != price_column_name]

# Exclude the time-based features from scaling as they are more like categorical/ordinal features
# and often don't benefit from min-max scaling or are handled differently (e.g., one-hot encoding if truly categorical)
time_based_features = ['year', 'month', 'day_of_week', 'day_of_year', 'week_of_year', 'quarter']
features_to_scale = [f for f in features_to_scale if f not in time_based_features]

print(f"\nFeatures identified for scaling: {features_to_scale}")

if features_to_scale:
    # Initialize MinMaxScaler
    scaler = MinMaxScaler()

    # Fit and transform the selected features
    # IMPORTANT NOTE for future steps:
    # In a real machine learning pipeline, you should fit this scaler ONLY on your TRAINING data
    # to prevent "data leakage." Then, you apply the *fitted* scaler to both your training and test data.
    # For now, we are applying it directly to the full 'df' for preprocessing demonstration.
    df[features_to_scale] = scaler.fit_transform(df[features_to_scale])
    print("Numerical features scaled using MinMaxScaler.")
else:
    print("No numerical features identified for scaling (excluding target and time-based features).")

print("\n--- First 5 rows after scaling (showing scaled features) ---")
print(df.head())

print("\n--- Final DataFrame Info after all preprocessing steps ---")
print(df.info())
print("\n--- Final Missing Values Count ---")
print(df.isnull().sum())
print(f"Final DataFrame shape: {df.shape}")


Features identified for scaling: ['1h', '24h', '7d', '24h_volume', 'mkt_cap']
Numerical features scaled using MinMaxScaler.

--- First 5 rows after scaling (showing scaled features) ---
                      coin  symbol         price        1h       24h  \
date                                                                   
2022-03-16         Bitcoin     BTC  4.085946e+04  0.908636  0.552739   
2022-03-16  Iron Bank EURO   IBEUR  1.080000e+00  0.881101  0.524939   
2022-03-16       Prometeus    PROM  7.960000e+00  0.902378  0.534751   
2022-03-16    MaidSafeCoin    MAID  2.949200e-01  0.909887  0.536386   
2022-03-16    Bezoge Earth  BEZOGE  3.051000e-09  0.896120  0.524121   

                  7d    24h_volume   mkt_cap  year  month  day_of_week  \
date                                                                     
2022-03-16  0.118660  6.108705e-01  0.993446  2022      3            2   
2022-03-16  0.109756  1.644225e-06  0.000083  2022      3            2   
2022-03-16  

In [11]:
import os

# Define the path to the processed data directory
processed_data_path = '../data/processed/'
output_file_name = 'preprocessed_crypto_data.csv'
output_file_path = os.path.join(processed_data_path, output_file_name)

# Create the processed data directory if it doesn't exist
os.makedirs(processed_data_path, exist_ok=True)

# Save the preprocessed DataFrame to a CSV file
# We'll save the index as well since it contains our datetime values
try:
    df.to_csv(output_file_path, index=True)
    print(f"\nPreprocessed data successfully saved to: {output_file_path}")
except Exception as e:
    print(f"Error saving preprocessed data: {e}")

# You can optionally verify by trying to load it back
# verify_df = pd.read_csv(output_file_path, index_col=0, parse_dates=True)
# print("\nVerification: First 5 rows of the saved data:")
# print(verify_df.head())
# print("\nVerification: Info of the saved data:")
# print(verify_df.info())


Preprocessed data successfully saved to: ../data/processed/preprocessed_crypto_data.csv
